# Willhaben - der österreichische Gebrauchtwagenmarkt

## Projekt Vorstellung

Willhaben.at kennt jeder (zumindest in Österreich)! :)

Willhaben.at ist eine der größten Marktplattformen in Österreich, auf denen Gebrauchtwagen gehandelt werden. Im Durchschnitt sind ca. 100.000 Autos gelistet (Zeitraum Oktober-November 2022). 

Unser Projekt verfolgt das Ziel einen Snapshot des Marktes zu machen und den Einfluss verschiedener Faktoren auf den Preis eines Autos zu analysieren. Die Fragestellungen sind nicht weltbewegend, aber die gesammelten Daten enthalten eine große Variabilität und sind eine ideale Übung, um Python zu lernen.



## 0. Bevor wir starten...

Wir erklären die Datenbeschaffung Schritt für Schritt im Jupyter Notebook, allerdings verweisen wir auch auf die Existenz folgender Python Skripte (machen das gleiche, nur ohne Notebook Overhead).

```sh
## in Shell ausführen:
$ python scripts/01scrapper.py
$ python scripts/02json_to_csv.py
$ python scripts/03validation.py
```

## 1. Web Scrapping

Willhaben stellt die Daten nicht als freien Download zur Verfügung (zB in Form einer CSV Datei), sondern arbeitet sie als Inserat auf der Page ein. Um die Daten zu erhalten, müssen wir die Daten programmatisch extrahieren ("scrappen"). 

!["Inserat"](./assets/inserat.png)


Als nächstes schauen wir uns die URL an, die folgende Struktur aufweist:

- https://www.willhaben.at/iad/gebrauchtwagen/auto/gebrauchtwagenboerse?page=50&rows=90&PRICE_TO=25000

Über Query Parameters (nach dem "?" in der URL) lässt sich steuern, welche Anzeigen auf Willhaben sichtbar sind. Man kann bis zu 999 Seiten anzeigen lassen (`page=999`). Über die Filter Funktion, zB "Preis bis zu", erfahren wir, dass `PRICE_TO` ebenfalls ein Parameter ist. Dieser ist erforderlich, um die Daten zu segmentieren, falls man mit dem theoretischen Limit von `page=999` und `rows=90` nicht auskommt (89.910 Inserate).

Der nächste Schritt ist zu analysieren, wie die Seite von Willhaben aufgebaut ist. Dazu schauen wir uns den Quellcode der HTML Seite an und stellen fest, dass tatsächlich die Daten als JSON mitgeliefert werden! Hintergrund ist der, dass Willhaben.at mit dem Web Framework Next.js aufgebaut ist: Der Browser ruft die URL auf, erhält JSON Daten und über JavaScript werden einzelne HTML Elemente dynamisch ausgetauscht. 

Im HTML Code ist das JSON hier zu finden: 
```html
<script id="__NEXT_DATA__" type="application/json">
  // json data...
</script>
```

Damit haben wir alle Wissenskomponenten, die wir brauchen, um die Daten zu extrahieren. Time to code!




Zum Scrappen, fahren wir keine schweren Geschütze auf (e.g. Playwright), sondern nutzen die Python Library `requests`. Außerdem definieren wir zwei eigene Exceptions, falls ein Request schief geht oder das JSON Fehler enthält.

In [1]:
import requests
import json
import time
from datetime import datetime
from typing import Dict

class JSONDataError(Exception):
    pass

class RequestNotSuccessfulError(Exception):
    pass

Requests ist sehr einfach, man schickt eine URL als Input und erhält eine Response als Output:

In [5]:
r = requests.get("https://www.willhaben.at/iad/gebrauchtwagen/auto/gebrauchtwagenboerse?page=50&rows=90&PRICE_TO=25000")
print(f"HTTP Status code: {r.status_code}")
# r.text        # der komplette HTML Code als Text

HTTP Status code: 200


Als Nächstes definieren wir eine Funktion, die mit Query Parameters gefüttert wird und als Output die Response des Willhaben Servers im Text Format schickt. Innerhalb der Funktion wird die URL dynamisch konstruiert und ein paar Cookies mitgeschickt (DSVGO Consent, etc.).

In [6]:
def get_html_from_willhaben(page: int = 1, price_to: int = 0, price_from: int = 0) -> str:
    """
    Scrap Willhaben Gebrauchtwagenboerse and return page html as string 
    """
    url = "https://www.willhaben.at/iad/gebrauchtwagen/auto/gebrauchtwagenboerse"
    cookies = {
        "IADVISITOR": "a58cf0e4-74a9-40c3-bf44-5fb0b9a714bc",
        "context": "prod",
        "TRACKINGID": "75349b84-607f-4161-9bc4-99f8aeddb123",
        "x-bbx-csrf-token": "5a4f4a0d-f121-4146-9f59-32ebcac0663d",
        "SRV": "1|Y0bcV",
        "didomi_token": "eyJ1c2VyX2lkIjoiMTgzY2NjY2EtYTMzZS02ZTA4LTkxOGYtMDNkOWYxYTIzM2U1IiwiY3JlYXRlZCI6IjIwMjItMTAtMTJUMTU6MjU6MTEuMjEyWiIsInVwZGF0ZWQiOiIyMDIyLTEwLTEyVDE1OjI1OjExLjIxMloiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiYW1hem9uIiwiZ29vZ2xlIiwiYzpvZXdhLVhBUW1HU2duIiwiYzphbWF6b24tbW9iaWxlLWFkcyIsImM6aG90amFyIiwiYzp1c2Vyem9vbSIsImM6YW1hem9uLWFzc29jaWF0ZXMiLCJjOnh4eGx1dHprLW05ZlFrUHRMIiwiYzpvcHRpb25hbGUtYm5BRXlaeHkiXX0sInB1cnBvc2VzIjp7ImVuYWJsZWQiOlsieHh4bHV0enItcWtyYXAzM1EiLCJnZW9sb2NhdGlvbl9kYXRhIiwiZGV2aWNlX2NoYXJhY3RlcmlzdGljcyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSIsImM6d2lsbGhhYmVuLVpxR242WXh6Il19LCJ2ZXJzaW9uIjoyLCJhYyI6IkFrdUFFQUZrQkpZQS5Ba3VBQ0FrcyJ9",
        "RANDOM_USER_GROUP_COOKIE_NAME": "48",
        "euconsent-v2": "CPgvCMAPgvCMAAHABBENCkCsAP_AAH_AAAAAG9tf_X_fb2_j-_59f_t0eY1P9_7_v-0zjhedk-8Nyd_X_L8X52M7vB36pq4KuR4ku3LBAQdlHOHcTQmw6IkVqSPsbk2Mr7NKJ7PEmlMbOydYGH9_n1XT-ZKY79__f_7z_v-v___37r__7-3f3_vp9V-BugBJhq3EAXYljgTbRhFAiBGFYSHQCgAooBhaIDCAlcFOyuAn1hAgAQCgCMCIEOAKMGAQAAAQBIREAIEeCAAAEQCAAEACoRCAAjQBBQASBgEAAoBoWAEUAQgSEGRARFKYEBEiQUE8gQglB_oYYQh1FAAA.f_gAD_gAAAAA",
        "_pbjs_userid_consent_data": "3265244935257896",
        "COUNTER_FOR_ADVERTISING_FIRST_PARTY_UID_V2": "0",
    }
    params = {"rows": 75}

    if page > 0 and type(page) == int:
        params.update({"page": page})

    if price_to > 0 and type(price_to) == int:
        params.update({"PRICE_TO": price_to})

    if price_from > 0 and type(price_from) == int:
        params.update({"PRICE_FROM": price_from})

    r = requests.get(url, cookies=cookies, params=params, timeout=60)

    if not r.ok:
        raise RequestNotSuccessfulError("Request was not successful", page)

    return r.text
